![T2](https://techcrunch.com/wp-content/uploads/2009/02/terminator.jpg)

In [18]:
import string     # Provides Easy Access to Character Sets
import random     # Used for Picking Characters out of a list or string
import time       # Used to time how long the password cracking functions are taking
import subprocess # Used to send commands to the underlying shell
import crypt      # Used to generate Sha512 hash for Linux password cracking
import itertools  # Used to Calculate Permutations
try:from IPython.display import clear_output # Used make time updates dynamic without scrolling
except:pass # Module Not a Hard Requeirment

# Password Generator

In [2]:
char_set = string.ascii_letters + string.digits + '!@#$%^&*()'

In [3]:
print('char_set =', char_set)

char_set = abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*()


In [4]:
print(len(char_set))

72


In [5]:
def gen_password(length, char_set=None):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    password = ''
    for i in range(length):
        x = random.choice(char_set)
        password = password + x
    return password

In [6]:
my_pass = gen_password(8)
print(f'Generated password = {my_pass}')
print(f'Generated password = {gen_password(8)}')
print(f'Generated password = {gen_password(8)}')


Generated password = czObJJhv
Generated password = T2hjNflZ
Generated password = SS5VeC*Y


# Password/PIN Cracker

In [7]:
password_to_crack = '4721'

In [ ]:
count = 1
start_time = time.time()
show = True
while True:
    attempt = gen_password(len(password_to_crack), string.digits)
    if attempt == password_to_crack:
        print(f'Cracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
        break
    else:
        pass
        try:
            if show:
                clear_output(wait=True)
                print(f'Guess = {attempt} Attempt Number = {count:,} Time Taken = {time.time()-start_time:.4f} seconds')
        except:pass
    count = count+1

Guess = 3166 Attempt Number = 5,211 Time Taken = 13.6588 seconds


# Calculating Permutations with Repetition

*Formula:* 

- **N**^**R** = Possible_Outcomes
    - **N** is the number of characters 
    - **R** is repetition count, eg password length


For example a 4 digit PIN:

- N = 10 (0,1,2,3,4,5,6,7,8,9)
- R = 4
    - 10^4 = 10,000 possible outcomes

In [ ]:
char_set_len = 72
length = 8
print(f'Possilbe Permutations {char_set_len**length:,}')

# Sequential Cracking Method

In [ ]:
def crack_pass(password_to_crack, start_length=1, up_to_length=12, char_set=None, show=False):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    print(f'''
    Attempting to crack password up to {up_to_length} characters long using these charecters:
    {char_set}
    ''')
    cracked = False
    count = 1
    start_time = time.time()
    for length in range(start_length, up_to_length+1):
        if cracked != True:
            print(f'\tTrying with Password Length of {length} @ {time.time()-start_time:.4f} seconds')
            for i in itertools.product(char_set, repeat=length):
                attempt = ''.join(i)
                if attempt == password_to_crack:
                    print(f'\tCracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
                    cracked = True
                    break
                else:
                    pass
                    if show:
                        try:
                            clear_output(wait=True)
                            print(f'Guess = {attempt} Attempt Number = {count} Time Taken = {time.time()-start_time:.4f} seconds')
                        except:pass
                count = count+1
    if cracked:
        return 'Crack Success'
    else:
        return 'Crack Failed'

In [ ]:
#password_to_crack = 'C@t5'
password_to_crack = 'T2'
password_to_crack

In [ ]:
crack_pass(password_to_crack, show=True)

Guess = Sf Attempt Number = 3246 Time Taken = 7.0001 seconds


# Cracking a Linux Password

### Add a User and Change their Password
#### `sudo useradd testuser`
#### `sudo passwd testuser`

In [ ]:
def find_user(user):
    '''Find a user in /etc/shadow and return the salt and the hash'''
    try:
        output = subprocess.run('sudo cat /etc/shadow'.split(), capture_output=True)
        for line in output.stdout.decode().split('\n'):
            if user in line:
                print(f'-> Found User "{user}"')
                print(line)
                break
        salt = line.split(':')[1].split('$')[2]
        hashed = line.split(':')[1].split('$')[3]
        #print('\n',salt, hashed)
        return salt,hashed
    except:
        print(f'Could not find user {user} in /etc/shadow')
        return None,None

In [ ]:
def gen_hash_sha512(password, salt):
    '''Generate our Hash Output for Password with given Salt'''
    sha512_hash = crypt.crypt(password, f'$6${salt}')
    return sha512_hash

In [ ]:
def crack_hashed_pass(password_hash, salt, up_to_length=13, char_set=None, show=False):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    print(f'''
    Attempting to crack password up to {up_to_length} characters long using these charecters:
    {char_set}
    ''')
    cracked = False
    count = 1
    start_time = time.time()
    for length in range(up_to_length):
        if cracked != True:
            print(f'\tTrying with Password Length of {length} @ {time.time()-start_time:.4f} seconds')
            for i in itertools.product(char_set, repeat=length):
                attempt = ''.join(i)
                # Pass Attempt to gen_hash_sha512 function
                hashed_attempt = gen_hash_sha512(attempt, salt)
                if show:
                        clear_output(wait=True)
                        print(f'Input Salt = {salt}')
                        print(f'Input Hash = {password_hash}')
                        print(f'Guess = {attempt} Attempt Number = {count:,} Time Taken = {time.time()-start_time:.4f} seconds')
                        print(f'Guess Hash = {hashed_attempt}')
                if hashed_attempt == password_hash:
                    print(f'\tCracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
                    cracked = True
                    break
                count = count+1
    if cracked:
        return ('Crack Success', attempt)
    else:
        return 'Crack Failed'

In [ ]:
def crack_linux_password(username, show=False):
    # Find Encrypted Password Line in /etc/shadow
    salt, hashed = find_user(username)
    if salt == None:return
    complete = f'$6${salt}${hashed}'
    print(f'\nSalt = {salt}\nHash = {hashed}')
    # Pass Salt and Hashed Password into crack_hashed_pass function
    result = crack_hashed_pass(complete, salt, up_to_length=13, char_set=None, show=show)
    return result

In [17]:
crack_linux_password('testuser', show=True)

Input Salt = 72AqZYt34ogSYVN.
Input Hash = $6$72AqZYt34ogSYVN.$Xdrff364dYB0qwtR7hnwDFe3CTo2g20vp0kkhG3IVpWe2ntgQI7xGvmHouG/qAtnItqBSzG9hnpIGjki5xJJt/
Guess = T2 Attempt Number = 3,368 Time Taken = 29.9410 seconds
Guess Hash = $6$72AqZYt34ogSYVN.$Xdrff364dYB0qwtR7hnwDFe3CTo2g20vp0kkhG3IVpWe2ntgQI7xGvmHouG/qAtnItqBSzG9hnpIGjki5xJJt/
	Cracked Password "T2" on Try 3,368 and took 29.9410 seconds


('Crack Success', 'T2')

### Delete the user when finished
#### `sudo userdel testuser`